In [1]:
!pip install sentence-transformers chromadb langchain langchain-community pypdf transformers==4.45.2 huggingface-hub==0.33.4 accelerate --quiet

In [2]:
!pip install jupyterlab_widgets

In [3]:
import torch
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration

model_name = "Qwen/Qwen2-VL-2B-Instruct"

print("Model indiriliyor...")

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    attn_implementation="sdpa"
).to("cuda")

processor = AutoProcessor.from_pretrained(model_name)
processor.tokenizer.pad_token = processor.tokenizer.eos_token

print("Model ve işlemci yüklendi!")


C:\Users\Berat\anaconda3\envs\qwen2clean\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model indiriliyor...


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██| 2/2 [00:01<00:00,  1.27it/s]


Model ve işlemci yüklendi!


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

PDF_PATH = "kalori_cetveli_dataset.pdf"

loader = PyPDFLoader(PDF_PATH)
docs = loader.load()

print(f"PDF'ten yüklenen sayfa/doküman: {len(docs)}")

splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=80)
chunks = splitter.split_documents(docs)

print(f"Oluşturulan toplam chunk sayısı: {len(chunks)}")

# Örnek chunk
print("\nİlk chunk örneği:")
print(chunks[0].page_content[:300], "...")


📄 PDF'ten yüklenen sayfa/doküman: 13
🔪 Oluşturulan toplam chunk sayısı: 44

İlk chunk örneği:
1) Yulaf Ezmesi 
Kalori: 389 kcal (100g) 
 Kategori: Karbonhidrat 
 Açıklama: Lif oranı yüksek, uzun süre tokluk sağlar. Kompleks karbonhidrat kaynağıdır. 
2) Muz 
Kalori: 89 kcal (1 adet) 
 Kategori: Meyve 
 Açıklama: Doğal enerji sağlar, potasyum açısından zengindir. 
3) Süt (Yağsız) 
Kalori: 42 k ...


In [5]:
!pip install langchain_huggingface langchain-community --quiet

In [6]:
!pip install chromadb --quiet

In [7]:
!pip install --upgrade langchain --quiet

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import os

CHROMA_DIR = "./kalori_chroma"

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

if os.path.exists(CHROMA_DIR):
    print("Var olan ChromaDB yükleniyor...")
    vectordb = Chroma(
        persist_directory=CHROMA_DIR,
        embedding_function=embeddings
    )
else:
    print("Yeni ChromaDB oluşturuluyor...")
    vectordb = Chroma(
    persist_directory=CHROMA_DIR,
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "cosine"},  
    )
    vectordb.persist()

print("ChromaDB hazır!")

C:\Users\Berat\AppData\Local\Temp\ipykernel_31084\1586154632.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Var olan ChromaDB yükleniyor...


C:\Users\Berat\AppData\Local\Temp\ipykernel_31084\1586154632.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectordb = Chroma(


ChromaDB hazır!


In [9]:
def fetch_context(query, k=7):
    docs = vectordb.similarity_search(query, k=k)
    return "\n".join(d.page_content for d in docs)

In [10]:
import time

# --- SYSTEM PROMPT'U GÜÇLENDİRİYORUZ ---
SYSTEM_PROMPT = """
Sen yardımsever ve samimi bir diyetisyensin.
GÖREVİN: Kullanıcının sorusuna, veritabanından gelen yiyecekleri kullanarak TEK BİR PARAGRAF halinde cevap vermek.

KESİN KURALLAR:
1. FORMAT: Asla alt alta liste yapma, madde işareti koyma. Sanki karşındakiyle konuşuyormuş gibi akıcı bir metin yaz.
2. İÇERİK: Seçtiğin yiyeceklerin kalorilerini mutlaka cümle içinde belirt. Örneğin: "Yanına 150 kalorilik midye dolma alabilirsin" gibi.
3. ALAKASIZ VERİYİ ELE: Sadece soruyla uyumlu (örneğin hepsi bir öğle yemeği olabilecek) ürünleri seç. Hepsini saymak zorunda değilsin.
4. BAĞLAM: Yiyecekleri birbirine bağla. "Bunun yanına şu yakışır", "İçecek olarak da şunu ekleyelim" gibi ifadeler kullan.

ÖRNEK DOĞRU CEVAP:
"Senin için harika bir öğle menüsü hazırladım; ana yemek olarak 150 kalori değerindeki midye dolmayı tercih edebilirsin. Bunun yanına hafif olması açısından 85 kalorilik lor peyniri ekleyerek protein dengesini sağlayabilirsin. İçecek olarak ise sadece 13 kalori olan şekersiz badem sütü tüketirsen hem hafif hem de doyurucu bir öğün yapmış olursun."
"""

def rag_chat(question, k=5):
    # k=5 iyidir, çok veri gelirse modelin kafası karışıp listeye döner.
    context = fetch_context(question, k=k)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            # Modele veriyi verirken de nasıl kullanacağını hatırlatıyoruz
            "content": f"Soru: {question}\n\nKullanabileceğin Besinler (Bunlardan mantıklı bir menü oluştur ve kalorilerini cümle içinde geçir):\n{context}"
        }
    ]

    text = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = processor(
        text=[text],
        return_tensors="pt",
        padding=True
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        do_sample=True,
        temperature=0.6, # 0.6 idealdır, cümleleri bağlaması için esneklik sağlar.
        top_p=0.9,
        max_new_tokens=250, 
        repetition_penalty=1.2 # Tekrarı önler
    )

    outputs = [
        out_ids[len(inp_ids):]
        for inp_ids, out_ids in zip(inputs.input_ids, outputs)
    ]
    return processor.batch_decode(outputs, skip_special_tokens=True)[0]

In [11]:
print(rag_chat("500 kalorilik öğle menüsü öner."))

Bu gün senin için en iyi öğle menüsünü hazırlayıp sizin için 500 kalorik miktarda yer tutacağınız seçeneklerden herhangi biri seçerken, bu menünün kalorisi 500 kalsiyelere ulaşacak şekilde tasarlanmıştır:

- Hindistan Cevizi Yoğurdu - 85 kalori (100 gram)
- Yulaf Sütü - 60 kalori (100 ml)
- Badem Sütü (Şekersiz) - 13 kalori (100 ml)
- Lor Peyniri - 85 kalori (100 gr)
- Midye Dolma - 150 kalori (3 adet)

Sonuçta toplam 500 kalori alınacaktır. Bu menüler genellikle enerji kaynakları arasında yer alan besinler içerir ve aynı zamanda çeşitli kalorijeli ürünler ile temsil etmekte bulunurlar. Bu nedenle, bu menülere bağlı olarak farklı taraflardaki besinlerin kullanılması gerekebilir.


In [15]:
print(rag_chat("300 kalorilik öğle menüsü öner. Ama sağlıklı olsun önerin nedir."))

Bu öneri genellikle 300 kalorlik bir öğle menüsünü oluşturmaktadır:

Ana yemek: 150 kallarlık midye dolması
Karışımköpek: 85 kallarlık lork peyniri
Doğal tatlandırıcı: 34 kallarlık yeşil mercimek çorbası
Atıştırmalık: 85 kallarlık hindistan cevizi yogurtu
Bitkisel süt: 60 kallarlık fırında brokolia
Sebze: 33 kallarlık bamya pişmiş
Havuç: 18 kallarlık kêrez (haşlanmış)
Zeytinyağı: 119 kallarlık tahıl
Mantıksüz yağı: 119 kallarlık tahıl
Tatlý biber turşusu: 25 kallarlık tatlý biber turşusu
Badem sütü (şekersiz): 13 kallarlık badem sütü
Yeşil


In [16]:
print(rag_chat("Ben şişman mıyım"))

Ben şişman değilim çünkü bu bilgiyi öğrenmediniz. Ancak, genellikle şişmaların kalorinin yüksek olduğu için, her zaman sağlıklı bir şekilde beslenmenizi öneririm. Bu nedenle, sizin için en iyi seçenekler arasında yer alan limonlu yeşil çayı ve bal bulunmaktadır. Bu ürünler, metabolizmayı hızlandırdığını ve hafife enerjisi sağladığını gösteriyor. Ayrıca, zeytinyağı, makarna, maden suyu ve midye dolması gibi diğer ürünler de etkin bir şekilde enerjisini sağlayacakları için önemlidir. 

Sonuçta, ben Şişmanın yoktuğumuza inandırım ama, eğer başka bir şey varsa, size yardımcı olabilirim!


In [17]:
print(rag_chat("Midye Dolması sence nasıl yemeli miyim"))

Midye Dolması, deniz ürünü ile pirinç karışımına dayanan bir yiyecekdir. Bu yiyecek, genellikle 150 kalsiyel kalori değerinden oluşmaktadır. Ayrıca, bu yiyecek, karışımların yanı sıra düşük kalorilidir. 

Hardal, düşük kalorilidir ve sos türüdür. Bu yüzden, her iki seçenek arasında en az 10 kalsiyel kalori değerinin yer aldığı bir ayri yiyecek bulunmaz. Ancak, Hardal'ın kalori değeri 10 kalsiyeldir. 

Light Mayonez, normal mayoneze göre daha düşük kalorilidir. Bu nedenle, 40 kalsiyel kalori değerindeki bir ayri yiyecek olduğu söylenebilir. 

Yeşil Mercimek Çorbası, metabolizmayı hızlandıran bir içecek türüdür. Bu yüzden, 108 kalsiyel kalori değerindeki bir ayri yiyecek olduğunu düşünülmesi gerekmektedir. 

Bal, hafif
